In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
extracted = pd.read_excel(r'data/extracted_data.xlsx')
correction = pd.read_excel(r'data/agent_correction_requests.xlsx')

data = extracted.copy()
corrected = correction.copy()

In [191]:
class Data:
    def __init__(self, df1, df2):
        self.df1 = df1
        self.df2 = df2
    def clean_reference(self) -> pd.DataFrame:
        self.df1['Account Reference'] = self.df1['Account Reference'].str.replace('\'','')
        return self.df1

    def split_text(self) -> pd.DataFrame:
        note = self.df1.copy()
        notes = self.df1[self.df1['Notes'].str.len() > 100]
        notes.loc[:, 'Notes'] = notes.Notes.str[100:]
        note = pd.concat([note, notes])
        note.Notes = note.Notes.str[0:101]
        note = note.sort_index().reset_index(drop=True)
        return note

    def clean_data(self) -> pd.DataFrame:
        cleaned = self.df1.copy()
        corr = self.df2.copy().astype(str)
        corr['Date'] = corr['Date'] + " " + corr['Time']
        cleaned = cleaned[(cleaned['Date Gathered'].isin(corr['Date'])) & (cleaned['Account Reference'].isin(corr['Account number']))]

        for i in range(len(cleaned)):
            cleaned[corr.iloc[i,4]].iloc[i] = corr.iloc[i,5]
        self.df1.update(cleaned)
        return self.df1

    def correction_status(self) -> pd.DataFrame:
        cleaned = self.df1.copy()
        corr = self.df2.copy().astype(str)
        corr['Date'] = corr['Date'] + " " + corr['Time']
        cleaned = cleaned[(cleaned['Date Gathered'].isin(corr['Date'])) & (cleaned['Account Reference'].isin(corr['Account number']))]
        corr['Status'] = corr.apply(lambda x:"Corrected" if x['Date'] == cleaned['Date Gathered'] else "Not Found")
        return corr

    def save_data(self) -> None:
        return





In [192]:
mysample = Data(data, corrected)
mysample.clean_reference()
mysample.correction_status()


KeyError: 'Date'